
This notebook shows you how to create and query a table or DataFrame loaded from data stored in Azure Blob storage.


### Step 1: Set the data location and type

There are two ways to access Azure Blob storage: account keys and shared access signatures (SAS).

To get started, we need to set the location and type of the file.

In [0]:
storage_account_name = "haridatacontainer"
storage_account_access_key = "3***w=="

In [0]:
file_location = "wasbs://sourcedata@haridatacontainer.blob.core.windows.net/products.csv"
file_type = "csv"

In [0]:
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)


### Step 2: Read the data

Now that we have specified our file metadata, we can create a DataFrame. Notice that we use an *option* to specify that we want to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python.


Option 1 : Using "COPY INTO"
Pre-requisite : Generate shared access signature (SAS) tokensi n the Azure portal · Right-click the container or file and select Generate SAS from the drop-down menu

In [0]:
%sql 
DROP TABLE products_agg2;

CREATE TABLE IF NOT EXISTS products_agg2;

COPY INTO products_agg2
FROM 'wasbs://sourcedata@haridatacontainer.blob.core.windows.net/products.csv'
WITH (
  CREDENTIAL (AZURE_SAS_TOKEN = 'sp=XXX3D')
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true', 'mergeSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')
;


Option2 : Using "spark.read"

In [0]:
df = spark.read.format(file_type).option("inferSchema", "true").option("header","true").load(file_location)


### Step 3: Query the data

Now that we have created our DataFrame, we can query it. For instance, you can identify particular columns to select and display.

In [0]:
display(df.head(3))

In [0]:
df = df.withColumnRenamed("Category","BikeCategory")

In [0]:
display(df.head(3))


### Step 4: (Optional) Create a view or table

If you want to query this data as a table, you can simply register it as a *view* or a table.

In [0]:
df.createOrReplaceTempView("src_products")


We can query this view using Spark SQL. For instance, we can perform a simple aggregation. Notice how we can use `%sql` to query the view from SQL.

In [0]:
%sql

SELECT ProductName, AVG(ListPrice) FROM src_products GROUP BY ProductName ORDER BY AVG(ListPrice) DESC LIMIT 3


Since this table is registered as a temp view, it will be available only to this notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.

In [0]:
_sqldf.write.format("parquet").saveAsTable("products_agg")


This table will persist across cluster restarts and allow various users across different notebooks to query this data.

In [0]:
%sql 
SHOW CREATE TABLE default.products_agg

Write the transformed data back to Azure Data Lake Storage Gen2 for further processing(say via Azure Data Factory or Azure Synapse Analytics)

In [0]:
df.write.format("parquet").save("wasbs://sourcedata@haridatacontainer.blob.core.windows.net/transformed_products_delta")

In [0]:
df.write.format("delta").save("wasbs://sourcedata@haridatacontainer.blob.core.windows.net/transformed_products_delta")